In [1]:
import numpy as np

def my_nelder_mead(f, start_point, max_iter=100):
    alpha = 1  # reflection
    gamma = 2  # expansion
    rho = 0.5  # contraction
    sigma = 0.5  # shrink
    # make initial triangle (for 2D) or simplex (for higher dimensions)
    n = len(start_point)
    points = [start_point]
    for i in range(n):
        new_point = start_point.copy()
        new_point[i] = new_point[i] + 1
        points.append(new_point)
    points = np.array(points)
    # store history for visualization
    history = [points.copy()]
    # main loop
    for iter in range(max_iter):
        # sort points by function value
        values = [f(p) for p in points]
        order = np.argsort(values)
        points = points[order]
        # centroid of best points (all except worst)
        centroid = np.mean(points[:-1], axis=0)
        # try reflection
        reflected = centroid + alpha * (centroid - points[-1])
        reflected_val = f(reflected)
        if f(points[0]) <= reflected_val < f(points[-2]):
            points[-1] = reflected
        # try expansion
        elif reflected_val < f(points[0]):
            expanded = centroid + gamma * (reflected - centroid)
            if f(expanded) < reflected_val:
                points[-1] = expanded
            else:
                points[-1] = reflected
        # try contraction
        else:
            contracted = centroid + rho * (points[-1] - centroid)
            if f(contracted) < f(points[-1]):
                points[-1] = contracted
            else:
                # if nothing works, shrink everything towards best point
                for i in range(1, len(points)):
                    points[i] = points[0] + sigma * (points[i] - points[0])
        history.append(points.copy())
    return points[0], history  # return best point and history for visualization

# test it works
if __name__ == "__main__":
    def test_function(x):
        return (1-x[0])**2 + (100*(x[1] - x[0]**2)**2)
    
    start_point_x = np.random.randint(-1000,1000)
    start_point_y = np.random.randint(-1000,1000)
    start_point = np.array([start_point_x, start_point_y])
    
    result, history = my_nelder_mead(test_function, start_point)
    print(f"Found minimum at: {result}")
    print(f"Function value: {test_function(result)}")

Found minimum at: [-25 641]
Function value: 26276


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.animation import FuncAnimation
from matplotlib.colors import Normalize
from matplotlib.animation import PillowWriter
from mpl_toolkits.mplot3d import Axes3D

def create_optimization_video(f, history, filename='optimization.gif', interval=500):
    """
    Creates a comprehensive optimization animation with detailed function visualization
    and Nelder-Mead simplex movement.
    """
    history = np.array(history)
    
    # Calculate bounds with generous padding
    padding = 1.0
    min_vals = history.min(axis=(0, 1)) - padding
    max_vals = history.max(axis=(0, 1)) + padding
    
    # Create high-resolution mesh
    resolution = 300  # Increased resolution
    x = np.linspace(min_vals[0], max_vals[0], resolution)
    y = np.linspace(min_vals[1], max_vals[1], resolution)
    X, Y = np.meshgrid(x, y)
    
    # Calculate function values
    Z = np.array([[f([xi, yi]) for xi in x] for yi in y])
    
    # Create figure with two subplots: 3D surface and 2D contour
    fig = plt.figure(figsize=(20, 8))
    
    # 3D surface plot
    ax1 = fig.add_subplot(121, projection='3d')
    surf = ax1.plot_surface(X, Y, Z, cmap='magma', alpha=0.8)
    ax1.set_title("Objective Function Surface", pad=20, fontsize=14, fontweight='bold')
    ax1.set_xlabel("X coordinate", fontsize=12)
    ax1.set_ylabel("Y coordinate", fontsize=12)
    ax1.set_zlabel("Function Value", fontsize=12)
    
    # Optimize the view angle for better visualization
    ax1.view_init(elev=30, azim=45)
    
    # 2D contour plot for animation
    ax2 = fig.add_subplot(122)
    
    # Create detailed contour plot with multiple color schemes
    levels = np.linspace(Z.min(), Z.max(), 50)
    contour = ax2.contourf(X, Y, Z, levels=levels, cmap='viridis', alpha=0.9)
    
    # Add contour lines for better detail
    contour_lines = ax2.contour(X, Y, Z, levels=15, colors='white', alpha=0.3, linewidths=0.5)
    ax2.clabel(contour_lines, inline=True, fontsize=8, fmt='%.1f')
    
    # Add colorbar with improved styling
    cbar = plt.colorbar(contour, ax=ax2)
    cbar.ax.set_ylabel('Function Value', fontsize=10)
    
    # Enhanced plot styling
    ax2.set_title("Optimization Progress", pad=20, fontsize=14, fontweight='bold')
    ax2.set_xlabel("X coordinate", fontsize=12)
    ax2.set_ylabel("Y coordinate", fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # Initialize triangle with improved visibility
    triangle = Polygon(history[0], 
                      edgecolor='red',
                      fill=True,
                      alpha=0.4,
                      linewidth=2)
    ax2.add_patch(triangle)
    
    # Add information text box with enhanced styling
    text = ax2.text(0.02, 0.98, '',
                   transform=ax2.transAxes,
                   bbox=dict(facecolor='white', 
                           alpha=0.8, 
                           edgecolor='black',
                           boxstyle='round,pad=0.5'),
                   fontsize=10,
                   verticalalignment='top')
    
    def init():
        triangle.set_xy(history[0])
        text.set_text('')
        return triangle, text
    
    def update(frame):
        points = history[frame]
        triangle.set_xy(points)
        
        # Calculate current metrics
        function_values = [f(p) for p in points]
        best_idx = np.argmin(function_values)
        best_point = points[best_idx]
        worst_idx = np.argmax(function_values)
        
        # Detailed iteration information
        text.set_text(f"Iteration: {frame}\n"
                     f"Best value: {f(best_point):.4f}\n"
                     f"Best point: ({best_point[0]:.2f}, {best_point[1]:.2f})\n"
                     f"Simplex size: {np.linalg.norm(points[worst_idx] - best_point):.3f}")
        return triangle, text
    
    # Create smooth animation
    anim = FuncAnimation(fig, update,
                        frames=len(history),
                        init_func=init,
                        blit=True,
                        interval=interval)
    
    # Save with high quality settings
    writer = PillowWriter(fps=1000//interval)
    anim.save(filename, writer=writer, dpi=150)
    plt.close()
    
    print(f"Animation saved as {filename}")

# Example usage
def test_function(x):
    """
    Example objective function for testing.
    Replace this with your actual objective function.
    """
    return (x[0]**2 + x[1]**2)  # Simple quadratic function

create_optimization_video(test_function,history)

Animation saved as optimization.gif
